In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import os

import haversine as hs

from datetime import datetime, timedelta

In [2]:
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
race_info = pd.read_csv("Race_Info.csv")

race_info.shape

(1082, 15)

In [4]:
race_info.head(50)

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2023,4,Azerbaijan Grand Prix,2023-04-30,Azerbaijan,Baku,Baku City Circuit,baku,double,4,12989.08,40.3725,49.85330,1101,73
4,2023,5,Miami Grand Prix,2023-05-07,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
5,2023,6,Monaco Grand Prix,2023-05-28,Monaco,Monte-Carlo,Circuit de Monaco,monaco,double,3,7870.83,43.7347,7.42056,1104,6
6,2023,7,Spanish Grand Prix,2023-06-04,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,double,1,485.64,41.5700,2.26111,1105,4
7,2023,8,Canadian Grand Prix,2023-06-18,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7
8,2023,9,Austrian Grand Prix,2023-07-02,Austria,Spielberg,Red Bull Ring,red_bull_ring,double,2,6390.49,47.2197,14.76470,1107,70
9,2023,10,British Grand Prix,2023-07-09,UK,Silverstone,Silverstone Circuit,silverstone,double,1,1254.64,52.0786,-1.01694,1108,9


In [5]:
race_info['circuitRef'].value_counts()

monza             73
monaco            69
silverstone       58
spa               56
villeneuve        42
nurburgring       41
interlagos        40
hungaroring       38
red_bull_ring     37
hockenheimring    37
catalunya         33
zandvoort         33
suzuka            33
imola             30
albert_park       26
rodriguez         23
kyalami           20
galvez            20
bahrain           20
watkins_glen      20
sepang            19
magny_cours       18
ricard            18
shanghai          16
yas_marina        15
brands_hatch      14
marina_bay        14
estoril           13
americas          11
reims             11
adelaide          11
zolder            10
jacarepagua       10
jarama             9
istanbul           9
sochi              8
long_beach         8
mosport            8
detroit            7
jerez              7
baku               7
dijon              6
anderstorp         6
aintree            5
bremgarten         5
valencia           5
essarts            5
charade      

In [6]:
imola = race_info.loc[race_info['circuitRef'] == 'imola'].head(1)

print(type(imola))

imola

<class 'pandas.core.frame.DataFrame'>


,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
25,2022,4,Emilia Romagna Grand Prix,2022-04-24,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,single,2,16086.36,44.3439,11.7167,1077,21


In [7]:
shanghai = race_info.loc[race_info['circuitRef'] == 'shanghai'].head(1)

print(type(shanghai))

shanghai

<class 'pandas.core.frame.DataFrame'>


,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
85,2019,3,Chinese Grand Prix,2019-04-14,China,Shanghai,Shanghai International Circuit,shanghai,single,2,6804.82,31.3389,121.22,1012,17


In [8]:
f1_23 = race_info.loc[race_info['year'] == 2023]

f1_23

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2023,4,Azerbaijan Grand Prix,2023-04-30,Azerbaijan,Baku,Baku City Circuit,baku,double,4,12989.08,40.3725,49.85330,1101,73
4,2023,5,Miami Grand Prix,2023-05-07,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
5,2023,6,Monaco Grand Prix,2023-05-28,Monaco,Monte-Carlo,Circuit de Monaco,monaco,double,3,7870.83,43.7347,7.42056,1104,6
6,2023,7,Spanish Grand Prix,2023-06-04,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,double,1,485.64,41.5700,2.26111,1105,4
7,2023,8,Canadian Grand Prix,2023-06-18,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7
8,2023,9,Austrian Grand Prix,2023-07-02,Austria,Spielberg,Red Bull Ring,red_bull_ring,double,2,6390.49,47.2197,14.76470,1107,70
9,2023,10,British Grand Prix,2023-07-09,UK,Silverstone,Silverstone Circuit,silverstone,double,1,1254.64,52.0786,-1.01694,1108,9


In [9]:
f1_23.loc[:2]

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.5106,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.1044,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.9680,1100,1


In [10]:
f1_23.loc[3:4]

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
3,2023,4,Azerbaijan Grand Prix,2023-04-30,Azerbaijan,Baku,Baku City Circuit,baku,double,4,12989.08,40.3725,49.8533,1101,73
4,2023,5,Miami Grand Prix,2023-05-07,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.2389,1102,79


In [11]:
f1_23.loc[5:]

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
5,2023,6,Monaco Grand Prix,2023-05-28,Monaco,Monte-Carlo,Circuit de Monaco,monaco,double,3,7870.83,43.7347,7.42056,1104,6
6,2023,7,Spanish Grand Prix,2023-06-04,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,double,1,485.64,41.5700,2.26111,1105,4
7,2023,8,Canadian Grand Prix,2023-06-18,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7
8,2023,9,Austrian Grand Prix,2023-07-02,Austria,Spielberg,Red Bull Ring,red_bull_ring,double,2,6390.49,47.2197,14.76470,1107,70
9,2023,10,British Grand Prix,2023-07-09,UK,Silverstone,Silverstone Circuit,silverstone,double,1,1254.64,52.0786,-1.01694,1108,9
10,2023,11,Hungarian Grand Prix,2023-07-23,Hungary,Budapest,Hungaroring,hungaroring,double,2,1531.29,47.5789,19.24860,1109,11
11,2023,12,Belgian Grand Prix,2023-07-30,Belgium,Spa,Circuit de Spa-Francorchamps,spa,double,1,1017.59,50.4372,5.97139,1110,13
12,2023,13,Dutch Grand Prix,2023-08-27,Netherlands,Zandvoort,Circuit Park Zandvoort,zandvoort,double,4,238.60,52.3888,4.54092,1111,39
13,2023,14,Italian Grand Prix,2023-09-03,Italy,Monza,Autodromo Nazionale di Monza,monza,double,1,828.29,45.6156,9.28111,1112,14
14,2023,15,Singapore Grand Prix,2023-09-17,Singapore,Marina Bay,Marina Bay Street Circuit,marina_bay,double,2,10260.98,1.2914,103.86400,1113,15


In [12]:
f1_23_planned = pd.concat([f1_23.loc[:2], shanghai, f1_23.loc[3:4], imola, f1_23.loc[5:]]).reset_index(drop=True)

f1_23_planned

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2019,3,Chinese Grand Prix,2019-04-14,China,Shanghai,Shanghai International Circuit,shanghai,single,2,6804.82,31.3389,121.22000,1012,17
4,2023,4,Azerbaijan Grand Prix,2023-04-30,Azerbaijan,Baku,Baku City Circuit,baku,double,4,12989.08,40.3725,49.85330,1101,73
5,2023,5,Miami Grand Prix,2023-05-07,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
6,2022,4,Emilia Romagna Grand Prix,2022-04-24,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,single,2,16086.36,44.3439,11.71670,1077,21
7,2023,6,Monaco Grand Prix,2023-05-28,Monaco,Monte-Carlo,Circuit de Monaco,monaco,double,3,7870.83,43.7347,7.42056,1104,6
8,2023,7,Spanish Grand Prix,2023-06-04,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,double,1,485.64,41.5700,2.26111,1105,4
9,2023,8,Canadian Grand Prix,2023-06-18,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7


In [13]:
for index, row in f1_23_planned.iterrows():
    #print(index+1, row['circuitRef'], row['round'])
    f1_23_planned.loc[index, 'round'] = index + 1

f1_23_planned

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2019,4,Chinese Grand Prix,2019-04-14,China,Shanghai,Shanghai International Circuit,shanghai,single,2,6804.82,31.3389,121.22000,1012,17
4,2023,5,Azerbaijan Grand Prix,2023-04-30,Azerbaijan,Baku,Baku City Circuit,baku,double,4,12989.08,40.3725,49.85330,1101,73
5,2023,6,Miami Grand Prix,2023-05-07,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
6,2022,7,Emilia Romagna Grand Prix,2022-04-24,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,single,2,16086.36,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,2023-05-28,Monaco,Monte-Carlo,Circuit de Monaco,monaco,double,3,7870.83,43.7347,7.42056,1104,6
8,2023,9,Spanish Grand Prix,2023-06-04,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,double,1,485.64,41.5700,2.26111,1105,4
9,2023,10,Canadian Grand Prix,2023-06-18,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7


In [14]:
f1_23_planned = f1_23_planned.assign(year=2023)

f1_23_planned

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2023,4,Chinese Grand Prix,2019-04-14,China,Shanghai,Shanghai International Circuit,shanghai,single,2,6804.82,31.3389,121.22000,1012,17
4,2023,5,Azerbaijan Grand Prix,2023-04-30,Azerbaijan,Baku,Baku City Circuit,baku,double,4,12989.08,40.3725,49.85330,1101,73
5,2023,6,Miami Grand Prix,2023-05-07,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
6,2023,7,Emilia Romagna Grand Prix,2022-04-24,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,single,2,16086.36,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,2023-05-28,Monaco,Monte-Carlo,Circuit de Monaco,monaco,double,3,7870.83,43.7347,7.42056,1104,6
8,2023,9,Spanish Grand Prix,2023-06-04,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,double,1,485.64,41.5700,2.26111,1105,4
9,2023,10,Canadian Grand Prix,2023-06-18,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7


In [15]:
#f1_23_planned.loc[f1_23_planned['circuitRef'] == 'shanghai', 'date'] = '2023-04-16'

type(f1_23_planned.loc[f1_23_planned['circuitRef'] == 'shanghai', 'date'].values[0])

str

In [16]:
#type(pd.to_datetime('2023-04-16'))

In [17]:
f1_23_planned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           24 non-null     int64  
 1   round          24 non-null     int64  
 2   race           24 non-null     object 
 3   date           24 non-null     object 
 4   country        24 non-null     object 
 5   location       24 non-null     object 
 6   circuit        24 non-null     object 
 7   circuitRef     24 non-null     object 
 8   header         24 non-null     object 
 9   gap            24 non-null     int64  
 10  distance (km)  24 non-null     float64
 11  lat            24 non-null     float64
 12  lng            24 non-null     float64
 13  raceId         24 non-null     int64  
 14  circuitId      24 non-null     int64  
dtypes: float64(3), int64(5), object(7)
memory usage: 2.9+ KB


In [18]:
race_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1082 entries, 0 to 1081
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           1082 non-null   int64  
 1   round          1082 non-null   int64  
 2   race           1082 non-null   object 
 3   date           1082 non-null   object 
 4   country        1082 non-null   object 
 5   location       1082 non-null   object 
 6   circuit        1082 non-null   object 
 7   circuitRef     1082 non-null   object 
 8   header         1082 non-null   object 
 9   gap            1082 non-null   int64  
 10  distance (km)  1082 non-null   float64
 11  lat            1082 non-null   float64
 12  lng            1082 non-null   float64
 13  raceId         1082 non-null   int64  
 14  circuitId      1082 non-null   int64  
dtypes: float64(3), int64(5), object(7)
memory usage: 126.9+ KB


In [19]:
shanghai_date = datetime.strptime('2023-04-16', '%Y-%m-%d').date()

#print(type(shanghai))

print(shanghai_date)

2023-04-16


In [20]:
imola_date = datetime.strptime('2023-05-21', '%Y-%m-%d').date()

print(imola_date)

2023-05-21


In [21]:
f1_23_planned.loc[f1_23_planned['circuitRef'] == 'shanghai', 'date'] = shanghai_date

f1_23_planned.loc[f1_23_planned['circuitRef'] == 'imola', 'date'] = imola_date

In [22]:
f1_23_planned

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2023,4,Chinese Grand Prix,2023-04-16,China,Shanghai,Shanghai International Circuit,shanghai,single,2,6804.82,31.3389,121.22000,1012,17
4,2023,5,Azerbaijan Grand Prix,2023-04-30,Azerbaijan,Baku,Baku City Circuit,baku,double,4,12989.08,40.3725,49.85330,1101,73
5,2023,6,Miami Grand Prix,2023-05-07,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
6,2023,7,Emilia Romagna Grand Prix,2023-05-21,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,single,2,16086.36,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,2023-05-28,Monaco,Monte-Carlo,Circuit de Monaco,monaco,double,3,7870.83,43.7347,7.42056,1104,6
8,2023,9,Spanish Grand Prix,2023-06-04,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,double,1,485.64,41.5700,2.26111,1105,4
9,2023,10,Canadian Grand Prix,2023-06-18,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7


In [23]:
f = '%Y-%m-%d'

f1_23_planned['date'] = pd.to_datetime(f1_23_planned['date'], format=f).dt.date

f1_23_planned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           24 non-null     int64  
 1   round          24 non-null     int64  
 2   race           24 non-null     object 
 3   date           24 non-null     object 
 4   country        24 non-null     object 
 5   location       24 non-null     object 
 6   circuit        24 non-null     object 
 7   circuitRef     24 non-null     object 
 8   header         24 non-null     object 
 9   gap            24 non-null     int64  
 10  distance (km)  24 non-null     float64
 11  lat            24 non-null     float64
 12  lng            24 non-null     float64
 13  raceId         24 non-null     int64  
 14  circuitId      24 non-null     int64  
dtypes: float64(3), int64(5), object(7)
memory usage: 2.9+ KB


In [24]:
i = 0
for index, row in f1_23_planned.iterrows():
    #print(row['round'], row['date'], type(row['date']))
    
    if row['round'] == 1:
        gap = 0
        
        #print(row['year'])
        i = i+1
    else:
        d2 = row['date']
        d1 = f1_23_planned.loc[(index-1), 'date']
        
        m2 = d2 - timedelta(days=d2.weekday())
        m1 = d1 - timedelta(days=d1.weekday())
        
        gap = (m2-m1).days/7
        
    f1_23_planned.loc[index, 'gap'] = gap
#print(i)

#race_info['round'].value_counts()

f1_23_planned['gap'] = f1_23_planned['gap'].astype(int)

f1_23_planned

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2023,4,Chinese Grand Prix,2023-04-16,China,Shanghai,Shanghai International Circuit,shanghai,single,2,6804.82,31.3389,121.22000,1012,17
4,2023,5,Azerbaijan Grand Prix,2023-04-30,Azerbaijan,Baku,Baku City Circuit,baku,double,2,12989.08,40.3725,49.85330,1101,73
5,2023,6,Miami Grand Prix,2023-05-07,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
6,2023,7,Emilia Romagna Grand Prix,2023-05-21,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,single,2,16086.36,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,2023-05-28,Monaco,Monte-Carlo,Circuit de Monaco,monaco,double,1,7870.83,43.7347,7.42056,1104,6
8,2023,9,Spanish Grand Prix,2023-06-04,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,double,1,485.64,41.5700,2.26111,1105,4
9,2023,10,Canadian Grand Prix,2023-06-18,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7


In [25]:
for index, row in f1_23_planned.iterrows():
    year = row['year']
    last = f1_23_planned.loc[f1_23_planned['year'] == year, 'round'].values.max()
    
    if row['round'] == 1:
        if f1_23_planned.loc[(index+1), 'gap'] == 1:
            if f1_23_planned.loc[(index+2), 'gap'] == 1:
                header = "triple"
            else:
                header = "double"
        else:
            header = "single"
    
    elif row['round'] < last-1:
        if row['gap'] == 1:
            if (f1_23_planned.loc[(index+1), 'gap'] == 1) or (f1_23_planned.loc[(index-1), 'gap'] == 1):
                header = "triple"
            else:
                header = "double"
        
        elif f1_23_planned.loc[(index+1), 'gap'] == 1:
            if f1_23_planned.loc[(index+2), 'gap'] == 1:
                header = "triple"
            else:
                header = "double"
        else:
            header = "single"
            
    elif row['round'] == last-1:
        if row['gap'] == 1:
            if (f1_23_planned.loc[(index+1), 'gap'] == 1) or (f1_23_planned.loc[(index-1), 'gap'] == 1):
                header = "triple"
            else:
                header = "double"
                
        elif f1_23_planned.loc[(index+1), 'gap'] == 1:
            header = "double"
                
        else:
            header = "single"
    
    else:
        if row['gap'] == 1:
            if f1_23_planned.loc[(index-1), 'gap'] == 1:
                header = "triple"
            else:
                header = "double"
        else:
            header = "single"
    
    #print(row['round'], header)
    
    f1_23_planned.loc[index, 'header'] = header

In [26]:
f1_23_planned

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2023,4,Chinese Grand Prix,2023-04-16,China,Shanghai,Shanghai International Circuit,shanghai,single,2,6804.82,31.3389,121.22000,1012,17
4,2023,5,Azerbaijan Grand Prix,2023-04-30,Azerbaijan,Baku,Baku City Circuit,baku,double,2,12989.08,40.3725,49.85330,1101,73
5,2023,6,Miami Grand Prix,2023-05-07,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
6,2023,7,Emilia Romagna Grand Prix,2023-05-21,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,triple,2,16086.36,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,2023-05-28,Monaco,Monte-Carlo,Circuit de Monaco,monaco,triple,1,7870.83,43.7347,7.42056,1104,6
8,2023,9,Spanish Grand Prix,2023-06-04,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,triple,1,485.64,41.5700,2.26111,1105,4
9,2023,10,Canadian Grand Prix,2023-06-18,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7


In [27]:
f1_23_planned.loc[f1_23_planned['circuitRef'] == 'vegas', 'country'] = 'USA'

f1_23_planned

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2023,4,Chinese Grand Prix,2023-04-16,China,Shanghai,Shanghai International Circuit,shanghai,single,2,6804.82,31.3389,121.22000,1012,17
4,2023,5,Azerbaijan Grand Prix,2023-04-30,Azerbaijan,Baku,Baku City Circuit,baku,double,2,12989.08,40.3725,49.85330,1101,73
5,2023,6,Miami Grand Prix,2023-05-07,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
6,2023,7,Emilia Romagna Grand Prix,2023-05-21,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,triple,2,16086.36,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,2023-05-28,Monaco,Monte-Carlo,Circuit de Monaco,monaco,triple,1,7870.83,43.7347,7.42056,1104,6
8,2023,9,Spanish Grand Prix,2023-06-04,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,triple,1,485.64,41.5700,2.26111,1105,4
9,2023,10,Canadian Grand Prix,2023-06-18,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7


In [43]:
race_info['year'].unique().tolist()

[2023,
 2022,
 2021,
 2020,
 2019,
 2018,
 2017,
 2016,
 2015,
 2014,
 2013,
 2012,
 2011,
 2010,
 2009,
 2008,
 2007,
 2006,
 2005,
 2004,
 2003,
 2002,
 2001,
 2000,
 1999,
 1998,
 1997,
 1996,
 1995,
 1994,
 1993,
 1992,
 1991,
 1990,
 1989,
 1988,
 1987,
 1986,
 1985,
 1984,
 1983,
 1982,
 1981,
 1980,
 1979,
 1978,
 1977,
 1976,
 1975,
 1974,
 1973,
 1972,
 1971,
 1970,
 1969,
 1968,
 1967,
 1966,
 1965,
 1964,
 1963,
 1962,
 1961,
 1960,
 1959,
 1958,
 1957,
 1956,
 1955,
 1954,
 1953,
 1952,
 1951,
 1950]

In [44]:
race_history = race_info.loc[race_info['year'] < 2023]

race_history['year'].unique().tolist()

[2022,
 2021,
 2020,
 2019,
 2018,
 2017,
 2016,
 2015,
 2014,
 2013,
 2012,
 2011,
 2010,
 2009,
 2008,
 2007,
 2006,
 2005,
 2004,
 2003,
 2002,
 2001,
 2000,
 1999,
 1998,
 1997,
 1996,
 1995,
 1994,
 1993,
 1992,
 1991,
 1990,
 1989,
 1988,
 1987,
 1986,
 1985,
 1984,
 1983,
 1982,
 1981,
 1980,
 1979,
 1978,
 1977,
 1976,
 1975,
 1974,
 1973,
 1972,
 1971,
 1970,
 1969,
 1968,
 1967,
 1966,
 1965,
 1964,
 1963,
 1962,
 1961,
 1960,
 1959,
 1958,
 1957,
 1956,
 1955,
 1954,
 1953,
 1952,
 1951,
 1950]

In [47]:
f = '%Y-%m-%d'

race_history['date'] = pd.to_datetime(race_history['date'], format=f).dt.date

In [48]:
for index, row in race_history.iterrows():
        race_history.loc[index, 'month'] = race_history.loc[index, 'date'].month
        
race_history['month'] = race_history['month'].astype(int)

cols = ['year', 'round', 'race', 'date', 'month', 'country', 'location', 'circuit', 'circuitRef', 'header', 'gap', 'distance (km)', 'lat', 'lng', 'raceId', 'circuitId']

race_history = race_history[cols]

race_history.sample(20)

,year,round,race,date,month,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
318,2007,3,Bahrain Grand Prix,2007-04-15,4,Bahrain,Sakhir,Bahrain International Circuit,bahrain,double,1,6040.70,26.0325,50.51060,38,3
351,2005,3,Bahrain Grand Prix,2005-04-03,4,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,2,6040.70,26.0325,50.51060,73,3
97,2019,15,Singapore Grand Prix,2019-09-22,9,Singapore,Marina Bay,Marina Bay Street Circuit,marina_bay,double,2,10260.98,1.2914,103.86400,1024,15
722,1982,3,United States Grand Prix West,1982-04-04,4,USA,California,Long Beach,long_beach,single,2,10110.75,33.7651,-118.18900,469,43
1058,1953,6,British Grand Prix,1953-07-18,7,UK,Silverstone,Silverstone Circuit,silverstone,single,2,469.10,52.0786,-1.01694,813,9
529,1994,1,Brazilian Grand Prix,1994-03-27,3,Brazil,São Paulo,Autódromo José Carlos Pace,interlagos,single,0,0.00,-23.7036,-46.69970,257,18
546,1993,2,Brazilian Grand Prix,1993-03-28,3,Brazil,São Paulo,Autódromo José Carlos Pace,interlagos,single,2,7439.34,-23.7036,-46.69970,274,18
548,1993,4,San Marino Grand Prix,1993-04-25,4,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,single,2,1344.36,44.3439,11.71670,276,21
609,1989,1,Brazilian Grand Prix,1989-03-26,3,Brazil,Rio de Janeiro,Autódromo Internacional Nelson Piquet,jacarepagua,single,0,0.00,-22.9756,-43.39500,356,36
480,1997,2,Brazilian Grand Prix,1997-03-30,3,Brazil,São Paulo,Autódromo José Carlos Pace,interlagos,single,3,13062.93,-23.7036,-46.69970,208,18


In [45]:
tracks = f1_23_planned['circuitRef'].unique().tolist()

print(len(tracks))
print(tracks)

24
['bahrain', 'jeddah', 'albert_park', 'shanghai', 'baku', 'miami', 'imola', 'monaco', 'catalunya', 'villeneuve', 'red_bull_ring', 'silverstone', 'hungaroring', 'spa', 'zandvoort', 'monza', 'marina_bay', 'suzuka', 'losail', 'americas', 'rodriguez', 'interlagos', 'vegas', 'yas_marina']


In [66]:
race_history.loc[race_history['country'] == 'Canada']

,year,round,race,date,month,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
30,2022,9,Canadian Grand Prix,2022-06-19,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,double,1,8930.53,45.5000,-73.5228,1082,7
89,2019,7,Canadian Grand Prix,2019-06-09,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,6121.74,45.5000,-73.5228,1016,7
110,2018,7,Canadian Grand Prix,2018-06-10,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,6121.74,45.5000,-73.5228,995,7
131,2017,7,Canadian Grand Prix,2017-06-11,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,6121.74,45.5000,-73.5228,975,7
151,2016,7,Canadian Grand Prix,2016-06-12,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,double,2,6121.74,45.5000,-73.5228,954,7
172,2015,7,Canadian Grand Prix,2015-06-07,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,6121.74,45.5000,-73.5228,932,7
191,2014,7,Canadian Grand Prix,2014-06-08,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,6121.74,45.5000,-73.5228,906,7
210,2013,7,Canadian Grand Prix,2013-06-09,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,6121.74,45.5000,-73.5228,886,7
229,2012,7,Canadian Grand Prix,2012-06-10,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,6121.74,45.5000,-73.5228,866,7
249,2011,7,Canadian Grand Prix,2011-06-12,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,6121.74,45.5000,-73.5228,847,7


In [67]:
race_history.loc[race_history['country'] == 'Brazil']

,year,round,race,date,month,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
42,2022,21,Brazilian Grand Prix,2022-11-13,11,Brazil,São Paulo,Autódromo José Carlos Pace,interlagos,double,2,7430.98,-23.7036,-46.6997,1095,18
62,2021,19,São Paulo Grand Prix,2021-11-14,11,Brazil,São Paulo,Autódromo José Carlos Pace,interlagos,triple,1,7430.98,-23.7036,-46.6997,1071,18
102,2019,20,Brazilian Grand Prix,2019-11-17,11,Brazil,São Paulo,Autódromo José Carlos Pace,interlagos,single,2,8085.18,-23.7036,-46.6997,1029,18
123,2018,20,Brazilian Grand Prix,2018-11-11,11,Brazil,São Paulo,Autódromo José Carlos Pace,interlagos,single,2,7430.98,-23.7036,-46.6997,1008,18
143,2017,19,Brazilian Grand Prix,2017-11-12,11,Brazil,São Paulo,Autódromo José Carlos Pace,interlagos,single,2,7430.98,-23.7036,-46.6997,987,18
164,2016,20,Brazilian Grand Prix,2016-11-13,11,Brazil,São Paulo,Autódromo José Carlos Pace,interlagos,single,2,7430.98,-23.7036,-46.6997,967,18
183,2015,18,Brazilian Grand Prix,2015-11-15,11,Brazil,São Paulo,Autódromo José Carlos Pace,interlagos,single,2,7430.98,-23.7036,-46.6997,944,18
202,2014,18,Brazilian Grand Prix,2014-11-09,11,Brazil,São Paulo,Autódromo José Carlos Pace,interlagos,double,1,8085.18,-23.7036,-46.6997,917,18
222,2013,19,Brazilian Grand Prix,2013-11-24,11,Brazil,São Paulo,Autódromo José Carlos Pace,interlagos,double,1,8085.18,-23.7036,-46.6997,899,18
242,2012,20,Brazilian Grand Prix,2012-11-25,11,Brazil,São Paulo,Autódromo José Carlos Pace,interlagos,double,1,8085.18,-23.7036,-46.6997,879,18


In [55]:
for track in tracks:
    
    track_history = race_history.loc[race_history['circuitRef'] == track]
    print(track)
    print(track_history['month'].value_counts())
    print()

bahrain
4     12
3      5
11     1
12     1
Name: month, dtype: int64

jeddah
3     1
12    1
Name: month, dtype: int64

albert_park
3    23
4     2
Name: month, dtype: int64

shanghai
4     11
10     4
9      1
Name: month, dtype: int64

baku
6    4
4    2
Name: month, dtype: int64

miami
5    1
Name: month, dtype: int64

imola
4     17
5     11
11     1
9      1
Name: month, dtype: int64

monaco
5    63
6     5
Name: month, dtype: int64

catalunya
5    26
4     3
8     1
6     1
9     1
Name: month, dtype: int64

villeneuve
6     37
9      3
10     1
Name: month, dtype: int64

red_bull_ring
8    18
7    10
6     4
5     3
9     1
Name: month, dtype: int64

silverstone
7    50
6     3
8     2
4     1
5     1
Name: month, dtype: int64

hungaroring
8    25
7    12
Name: month, dtype: int64

spa
8    26
6    18
9     8
5     3
Name: month, dtype: int64

zandvoort
6    11
8    10
5     5
7     4
9     2
Name: month, dtype: int64

monza
9    72
Name: month, dtype: int64

marina_bay
9     1

In [69]:
print(race_history.shape[0])
print()
for i in range(1, 13):
    print(i, race_history.loc[race_history['month'] == i].shape[0])

1060

1 24
2 3
3 77
4 83
5 141
6 123
7 176
8 127
9 134
10 115
11 49
12 8


In [77]:
for year in range(1950, 2023):
    if year == 2020:
        continue
    
    race_year = race_history.loc[race_history['year'] == year]
    year_size = race_year.shape[0]
    print(year, '-', year_size)
    
    for i in range(1, 13):
        race_month = race_year.loc[race_year['month'] == i]
        month_size = race_month.shape[0]
        if month_size == 0:
            continue
        print(i, '-',month_size, '-', round((month_size/year_size)*100))
     
    print()

1950 - 6
5 - 2 - 33
6 - 2 - 33
7 - 1 - 17
9 - 1 - 17

1951 - 7
5 - 1 - 14
6 - 1 - 14
7 - 3 - 43
9 - 1 - 14
10 - 1 - 14

1952 - 7
5 - 1 - 14
6 - 1 - 14
7 - 2 - 29
8 - 2 - 29
9 - 1 - 14

1953 - 8
1 - 1 - 12
6 - 2 - 25
7 - 2 - 25
8 - 2 - 25
9 - 1 - 12

1954 - 8
1 - 1 - 12
6 - 1 - 12
7 - 2 - 25
8 - 2 - 25
9 - 1 - 12
10 - 1 - 12

1955 - 6
1 - 1 - 17
5 - 1 - 17
6 - 2 - 33
7 - 1 - 17
9 - 1 - 17

1956 - 7
1 - 1 - 14
5 - 1 - 14
6 - 1 - 14
7 - 2 - 29
8 - 1 - 14
9 - 1 - 14

1957 - 7
1 - 1 - 14
5 - 1 - 14
7 - 2 - 29
8 - 2 - 29
9 - 1 - 14

1958 - 10
1 - 1 - 10
5 - 2 - 20
6 - 1 - 10
7 - 2 - 20
8 - 2 - 20
9 - 1 - 10
10 - 1 - 10

1959 - 8
5 - 2 - 25
7 - 2 - 25
8 - 2 - 25
9 - 1 - 12
12 - 1 - 12

1960 - 9
2 - 1 - 11
5 - 1 - 11
6 - 2 - 22
7 - 2 - 22
8 - 1 - 11
9 - 1 - 11
11 - 1 - 11

1961 - 8
5 - 2 - 25
6 - 1 - 12
7 - 2 - 25
8 - 1 - 12
9 - 1 - 12
10 - 1 - 12

1962 - 9
5 - 1 - 11
6 - 2 - 22
7 - 2 - 22
8 - 1 - 11
9 - 1 - 11
10 - 1 - 11
12 - 1 - 11

1963 - 10
5 - 1 - 10
6 - 3 - 30
7 - 1 - 10
8 - 1 - 10
9 - 

In [64]:
for track in tracks:
    
    track_history = race_history.loc[race_history['circuitRef'] == track]
    
    print(track, track_history.shape[0])
    for i in range(1, 13):
        track_month = track_history.loc[track_history['month'] == i]
        
        if track_month.shape[0] == 0:
            continue
            
        print(i, '-', track_month.shape[0], '|', round((track_month.shape[0]/track_history.shape[0])*100, 2), "%")
    
    print()

bahrain 19
3 - 5 | 26.32 %
4 - 12 | 63.16 %
11 - 1 | 5.26 %
12 - 1 | 5.26 %

jeddah 2
3 - 1 | 50.0 %
12 - 1 | 50.0 %

albert_park 25
3 - 23 | 92.0 %
4 - 2 | 8.0 %

shanghai 16
4 - 11 | 68.75 %
9 - 1 | 6.25 %
10 - 4 | 25.0 %

baku 6
4 - 2 | 33.33 %
6 - 4 | 66.67 %

miami 1
5 - 1 | 100.0 %

imola 30
4 - 17 | 56.67 %
5 - 11 | 36.67 %
9 - 1 | 3.33 %
11 - 1 | 3.33 %

monaco 68
5 - 63 | 92.65 %
6 - 5 | 7.35 %

catalunya 32
4 - 3 | 9.38 %
5 - 26 | 81.25 %
6 - 1 | 3.12 %
8 - 1 | 3.12 %
9 - 1 | 3.12 %

villeneuve 41
6 - 37 | 90.24 %
9 - 3 | 7.32 %
10 - 1 | 2.44 %

red_bull_ring 36
5 - 3 | 8.33 %
6 - 4 | 11.11 %
7 - 10 | 27.78 %
8 - 18 | 50.0 %
9 - 1 | 2.78 %

silverstone 57
4 - 1 | 1.75 %
5 - 1 | 1.75 %
6 - 3 | 5.26 %
7 - 50 | 87.72 %
8 - 2 | 3.51 %

hungaroring 37
7 - 12 | 32.43 %
8 - 25 | 67.57 %

spa 55
5 - 3 | 5.45 %
6 - 18 | 32.73 %
8 - 26 | 47.27 %
9 - 8 | 14.55 %

zandvoort 32
5 - 5 | 15.62 %
6 - 11 | 34.38 %
7 - 4 | 12.5 %
8 - 10 | 31.25 %
9 - 2 | 6.25 %

monza 72
9 - 72 | 100.0 %

mari

In [28]:
for index, row in f1_23_planned.iterrows():
    #print(row['round'], row['location'])
    
    if row['round'] == 1:
        dist = 0
    else:
        lat1 = f1_23_planned.loc[(index-1), 'lat']
        lng1 = f1_23_planned.loc[(index-1), 'lng']
        
        lat2 = row['lat']
        lng2 = row['lng']
        
        loc2 = row['location']
        
        dist = round(hs.haversine((lat1, lng1), (lat2, lng2)), 2)
        
    #print(row['round'], row['location'], dist)
    f1_23_planned.loc[index, 'distance (km)'] = dist

In [29]:
f1_23_planned

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2023,4,Chinese Grand Prix,2023-04-16,China,Shanghai,Shanghai International Circuit,shanghai,single,2,8078.46,31.3389,121.22000,1012,17
4,2023,5,Azerbaijan Grand Prix,2023-04-30,Azerbaijan,Baku,Baku City Circuit,baku,double,2,6336.88,40.3725,49.85330,1101,73
5,2023,6,Miami Grand Prix,2023-05-07,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
6,2023,7,Emilia Romagna Grand Prix,2023-05-21,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,triple,2,8172.92,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,2023-05-28,Monaco,Monte-Carlo,Circuit de Monaco,monaco,triple,1,349.98,43.7347,7.42056,1104,6
8,2023,9,Spanish Grand Prix,2023-06-04,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,triple,1,485.64,41.5700,2.26111,1105,4
9,2023,10,Canadian Grand Prix,2023-06-18,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7


In [30]:
f1_23_planned.to_csv("F1_23_Planned.csv", index=False)

In [31]:
f1_23['distance (km)'].sum()

131513.22

In [32]:
f1_23_planned['distance (km)'].sum()

133591.55

In [33]:
f1_23_planned['circuitRef'].values.tolist()

['bahrain',
 'jeddah',
 'albert_park',
 'shanghai',
 'baku',
 'miami',
 'imola',
 'monaco',
 'catalunya',
 'villeneuve',
 'red_bull_ring',
 'silverstone',
 'hungaroring',
 'spa',
 'zandvoort',
 'monza',
 'marina_bay',
 'suzuka',
 'losail',
 'americas',
 'rodriguez',
 'interlagos',
 'vegas',
 'yas_marina']

In [34]:
f1_24_order = ['bahrain', 'jeddah', 'albert_park', 'suzuka', 'shanghai', 'miami', 'imola', 'monaco', 'villeneuve', 'catalunya', 'red_bull_ring', 'silverstone', 'hungaroring', 'spa', 'zandvoort','monza', 'baku', 'marina_bay', 'americas', 'rodriguez', 'interlagos', 'vegas', 'losail', 'yas_marina']

i = 1

for circuit in f1_24_order:
    print(i, circuit)
    i = i + 1
    
    
    
    

1 bahrain
2 jeddah
3 albert_park
4 suzuka
5 shanghai
6 miami
7 imola
8 monaco
9 villeneuve
10 catalunya
11 red_bull_ring
12 silverstone
13 hungaroring
14 spa
15 zandvoort
16 monza
17 baku
18 marina_bay
19 americas
20 rodriguez
21 interlagos
22 vegas
23 losail
24 yas_marina


In [35]:
f1_24 = pd.DataFrame()

for circuit in f1_24_order:
    row = f1_23_planned.loc[f1_23_planned['circuitRef'] == circuit]
    
    f1_24 = pd.concat([f1_24, row])
    
f1_24 = f1_24.reset_index(drop=True)

f1_24

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2023,18,Japanese Grand Prix,2023-09-24,Japan,Suzuka,Suzuka Circuit,suzuka,double,1,5035.99,34.8431,136.54100,1114,22
4,2023,4,Chinese Grand Prix,2023-04-16,China,Shanghai,Shanghai International Circuit,shanghai,single,2,8078.46,31.3389,121.22000,1012,17
5,2023,6,Miami Grand Prix,2023-05-07,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
6,2023,7,Emilia Romagna Grand Prix,2023-05-21,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,triple,2,8172.92,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,2023-05-28,Monaco,Monte-Carlo,Circuit de Monaco,monaco,triple,1,349.98,43.7347,7.42056,1104,6
8,2023,10,Canadian Grand Prix,2023-06-18,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7
9,2023,9,Spanish Grand Prix,2023-06-04,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,triple,1,485.64,41.5700,2.26111,1105,4


In [36]:
f1_24.drop(columns=['date', 'header', 'gap'], inplace=True)

f1_24

,year,round,race,country,location,circuit,circuitRef,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,Bahrain,Sakhir,Bahrain International Circuit,bahrain,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,12817.12,-37.8497,144.96800,1100,1
3,2023,18,Japanese Grand Prix,Japan,Suzuka,Suzuka Circuit,suzuka,5035.99,34.8431,136.54100,1114,22
4,2023,4,Chinese Grand Prix,China,Shanghai,Shanghai International Circuit,shanghai,8078.46,31.3389,121.22000,1012,17
5,2023,6,Miami Grand Prix,USA,Miami,Miami International Autodrome,miami,11015.93,25.9581,-80.23890,1102,79
6,2023,7,Emilia Romagna Grand Prix,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,8172.92,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,Monaco,Monte-Carlo,Circuit de Monaco,monaco,349.98,43.7347,7.42056,1104,6
8,2023,10,Canadian Grand Prix,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,5891.51,45.5000,-73.52280,1106,7
9,2023,9,Spanish Grand Prix,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,485.64,41.5700,2.26111,1105,4


In [37]:
for index, row in f1_24.iterrows():
    #print(index+1, row['circuitRef'], row['round'])
    f1_24.loc[index, 'round'] = index + 1

f1_24

,year,round,race,country,location,circuit,circuitRef,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,Bahrain,Sakhir,Bahrain International Circuit,bahrain,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,12817.12,-37.8497,144.96800,1100,1
3,2023,4,Japanese Grand Prix,Japan,Suzuka,Suzuka Circuit,suzuka,5035.99,34.8431,136.54100,1114,22
4,2023,5,Chinese Grand Prix,China,Shanghai,Shanghai International Circuit,shanghai,8078.46,31.3389,121.22000,1012,17
5,2023,6,Miami Grand Prix,USA,Miami,Miami International Autodrome,miami,11015.93,25.9581,-80.23890,1102,79
6,2023,7,Emilia Romagna Grand Prix,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,8172.92,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,Monaco,Monte-Carlo,Circuit de Monaco,monaco,349.98,43.7347,7.42056,1104,6
8,2023,9,Canadian Grand Prix,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,5891.51,45.5000,-73.52280,1106,7
9,2023,10,Spanish Grand Prix,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,485.64,41.5700,2.26111,1105,4


In [38]:
for index, row in f1_24.iterrows():
    #print(row['round'], row['location'])
    
    if row['round'] == 1:
        dist = 0
    else:
        lat1 = f1_24.loc[(index-1), 'lat']
        lng1 = f1_24.loc[(index-1), 'lng']
        
        lat2 = row['lat']
        lng2 = row['lng']
        
        loc2 = row['location']
        
        dist = round(hs.haversine((lat1, lng1), (lat2, lng2)), 2)
        
    #print(row['round'], row['location'], dist)
    f1_24.loc[index, 'distance (km)'] = dist

In [39]:
f1_24

,year,round,race,country,location,circuit,circuitRef,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,Bahrain,Sakhir,Bahrain International Circuit,bahrain,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,12817.12,-37.8497,144.96800,1100,1
3,2023,4,Japanese Grand Prix,Japan,Suzuka,Suzuka Circuit,suzuka,8129.72,34.8431,136.54100,1114,22
4,2023,5,Chinese Grand Prix,China,Shanghai,Shanghai International Circuit,shanghai,1477.81,31.3389,121.22000,1012,17
5,2023,6,Miami Grand Prix,USA,Miami,Miami International Autodrome,miami,13248.55,25.9581,-80.23890,1102,79
6,2023,7,Emilia Romagna Grand Prix,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,8172.92,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,Monaco,Monte-Carlo,Circuit de Monaco,monaco,349.98,43.7347,7.42056,1104,6
8,2023,9,Canadian Grand Prix,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,6121.74,45.5000,-73.52280,1106,7
9,2023,10,Spanish Grand Prix,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,5891.51,41.5700,2.26111,1105,4


In [40]:
f1_24['distance (km)'].sum()

121396.07

In [41]:
print("Planned 2023 :", f1_23_planned['distance (km)'].sum())
print("Actual 2024 :", f1_23['distance (km)'].sum())
print("Planned 2024 :", f1_24['distance (km)'].sum())

Planned 2023 : 133591.55
Actual 2024 : 131513.22
Planned 2024 : 121396.07
